In [1]:
from __future__ import print_function

In [2]:
from scipy.misc import imsave
import numpy as np
import time
from keras.applications import vgg16
from keras import backend as K

Using TensorFlow backend.


In [3]:
img_width = 128
img_height = 128

In [4]:
layer_name = 'block5_conv1'

In [5]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [6]:
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

Model loaded.


C:\Users\Yoogie\Anaconda3\envs\tensorflow\lib\site-packages\keras\applications\vgg16.py:182: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, None, None)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, None, None)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, None, None)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, None, None)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, None, None)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, None, None)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, None, None)   0         
__________

In [8]:
input_img = model.input

In [9]:
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


In [10]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [11]:

kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

# save the result to disk
imsave('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

Processing filter 0
Current loss value: 11.7226
Current loss value: 28.0194
Current loss value: 49.567
Current loss value: 75.705
Current loss value: 96.6414
Current loss value: 114.572
Current loss value: 134.393
Current loss value: 158.308
Current loss value: 176.726
Current loss value: 195.486
Current loss value: 215.688
Current loss value: 237.682
Current loss value: 260.095
Current loss value: 277.174
Current loss value: 297.379
Current loss value: 320.204
Current loss value: 336.968
Current loss value: 360.027
Current loss value: 381.012
Current loss value: 402.52
Filter 0 processed in 15s
Processing filter 1
Current loss value: 0.0
Filter 1 processed in 1s
Processing filter 2
Current loss value: 9.62418
Current loss value: 28.4785
Current loss value: 68.2935
Current loss value: 99.3468
Current loss value: 134.687
Current loss value: 168.339
Current loss value: 201.64
Current loss value: 237.894
Current loss value: 269.791
Current loss value: 299.358
Current loss value: 330.962
C

Current loss value: 123.489
Current loss value: 159.916
Current loss value: 204.821
Current loss value: 238.051
Current loss value: 281.541
Current loss value: 317.319
Current loss value: 358.323
Current loss value: 395.603
Current loss value: 431.16
Current loss value: 465.278
Current loss value: 500.154
Current loss value: 531.4
Current loss value: 565.2
Filter 15 processed in 13s
Processing filter 16
Current loss value: 0.0
Filter 16 processed in 1s
Processing filter 17
Current loss value: 0.901148
Current loss value: 19.7585
Current loss value: 38.7797
Current loss value: 83.2514
Current loss value: 135.221
Current loss value: 193.37
Current loss value: 253.433
Current loss value: 313.768
Current loss value: 380.828
Current loss value: 443.398
Current loss value: 519.081
Current loss value: 585.667
Current loss value: 656.576
Current loss value: 720.05
Current loss value: 782.803
Current loss value: 840.902
Current loss value: 908.167
Current loss value: 970.742
Current loss value:

Current loss value: 80.9082
Current loss value: 126.936
Current loss value: 183.442
Current loss value: 230.118
Current loss value: 292.191
Current loss value: 356.667
Current loss value: 422.101
Current loss value: 489.271
Current loss value: 548.27
Current loss value: 599.418
Current loss value: 655.92
Current loss value: 703.435
Current loss value: 745.549
Current loss value: 795.207
Current loss value: 843.676
Current loss value: 885.217
Current loss value: 936.364
Filter 34 processed in 12s
Processing filter 35
Current loss value: 0.0
Filter 35 processed in 1s
Processing filter 36
Current loss value: 11.1904
Current loss value: 26.0652
Current loss value: 94.2501
Current loss value: 175.353
Current loss value: 268.637
Current loss value: 344.978
Current loss value: 426.155
Current loss value: 492.911
Current loss value: 565.721
Current loss value: 632.891
Current loss value: 696.101
Current loss value: 750.519
Current loss value: 813.296
Current loss value: 867.878
Current loss va

Current loss value: 682.718
Current loss value: 757.77
Current loss value: 818.917
Current loss value: 885.887
Current loss value: 946.988
Current loss value: 1015.15
Current loss value: 1073.12
Current loss value: 1141.24
Current loss value: 1200.06
Current loss value: 1266.87
Filter 49 processed in 12s
Processing filter 50
Current loss value: 1.64738
Current loss value: 5.30334
Current loss value: 10.9724
Current loss value: 20.4439
Current loss value: 31.7464
Current loss value: 48.5575
Current loss value: 64.1274
Current loss value: 84.381
Current loss value: 104.427
Current loss value: 120.937
Current loss value: 152.775
Current loss value: 185.08
Current loss value: 216.797
Current loss value: 245.997
Current loss value: 276.313
Current loss value: 303.935
Current loss value: 334.149
Current loss value: 362.623
Current loss value: 394.031
Current loss value: 420.261
Filter 50 processed in 12s
Processing filter 51
Current loss value: 0.158742
Current loss value: 20.362
Current los

Current loss value: 0.924685
Current loss value: 15.2159
Current loss value: 37.6281
Current loss value: 60.4012
Current loss value: 95.7266
Current loss value: 130.098
Current loss value: 166.264
Current loss value: 197.896
Current loss value: 234.036
Current loss value: 277.988
Current loss value: 315.055
Current loss value: 364.534
Current loss value: 414.976
Current loss value: 462.716
Current loss value: 514.784
Current loss value: 563.336
Current loss value: 615.206
Current loss value: 666.008
Current loss value: 715.565
Current loss value: 775.702
Filter 70 processed in 12s
Processing filter 71
Current loss value: 0.0
Filter 71 processed in 1s
Processing filter 72
Current loss value: 0.0
Filter 72 processed in 1s
Processing filter 73
Current loss value: 26.9688
Current loss value: 61.6269
Current loss value: 102.867
Current loss value: 151.385
Current loss value: 195.13
Current loss value: 231.091
Current loss value: 281.636
Current loss value: 327.224
Current loss value: 369.04

...

Current loss value: 545.693
Current loss value: 590.31
Current loss value: 626.164
Current loss value: 671.846
Current loss value: 709.716
Current loss value: 751.653
Current loss value: 790.939
Filter 141 processed in 13s
Processing filter 142
Current loss value: 0.0
Filter 142 processed in 1s
Processing filter 143
Current loss value: 10.4466
Current loss value: 32.9447
Current loss value: 86.5276
Current loss value: 150.052
Current loss value: 229.021
Current loss value: 294.265
Current loss value: 362.847
Current loss value: 435.655
Current loss value: 508.31
Current loss value: 575.146
Current loss value: 647.527
Current loss value: 722.291
Current loss value: 793.316
Current loss value: 865.106
Current loss value: 936.345
Current loss value: 1005.55
Current loss value: 1066.34
Current loss value: 1131.62
Current loss value: 1195.43
Current loss value: 1257.36
Filter 143 processed in 14s
Processing filter 144
Current loss value: 0.151141
Current loss value: 2.0837
Current loss valu

Current loss value: 1262.7
Current loss value: 1334.32
Current loss value: 1405.38
Current loss value: 1473.85
Filter 157 processed in 13s
Processing filter 158
Current loss value: 7.53938
Current loss value: 10.2318
Current loss value: 29.2192
Current loss value: 56.664
Current loss value: 97.9886
Current loss value: 143.469
Current loss value: 195.735
Current loss value: 246.552
Current loss value: 299.489
Current loss value: 348.909
Current loss value: 399.563
Current loss value: 455.361
Current loss value: 507.728
Current loss value: 566.378
Current loss value: 618.573
Current loss value: 667.987
Current loss value: 714.512
Current loss value: 766.291
Current loss value: 818.544
Current loss value: 869.083
Filter 158 processed in 13s
Processing filter 159
Current loss value: 84.649
Current loss value: 98.1168
Current loss value: 128.135
Current loss value: 150.475
Current loss value: 172.661
Current loss value: 198.337
Current loss value: 223.648
Current loss value: 240.555
Current

Current loss value: 760.316
Current loss value: 807.022
Current loss value: 848.601
Current loss value: 890.964
Filter 174 processed in 13s
Processing filter 175
Current loss value: 1.11162
Current loss value: 25.9847
Current loss value: 82.6932
Current loss value: 135.756
Current loss value: 181.692
Current loss value: 236.051
Current loss value: 286.535
Current loss value: 335.88
Current loss value: 383.945
Current loss value: 442.27
Current loss value: 493.105
Current loss value: 545.733
Current loss value: 596.282
Current loss value: 647.563
Current loss value: 693.238
Current loss value: 740.808
Current loss value: 791.017
Current loss value: 839.547
Current loss value: 887.294
Current loss value: 929.437
Filter 175 processed in 13s
Processing filter 176
Current loss value: 0.0
Filter 176 processed in 2s
Processing filter 177
Current loss value: 7.70075
Current loss value: 23.7655
Current loss value: 49.7225
Current loss value: 86.9721
Current loss value: 125.771
Current loss valu

Current loss value: 515.71
Current loss value: 555.249
Current loss value: 592.94
Current loss value: 631.234
Filter 191 processed in 13s
Processing filter 192
Current loss value: 8.04926
Current loss value: 16.8354
Current loss value: 42.9116
Current loss value: 75.1287
Current loss value: 106.725
Current loss value: 140.481
Current loss value: 180.264
Current loss value: 219.481
Current loss value: 250.623
Current loss value: 286.28
Current loss value: 327.807
Current loss value: 365.176
Current loss value: 403.644
Current loss value: 444.791
Current loss value: 488.73
Current loss value: 524.916
Current loss value: 563.623
Current loss value: 600.345
Current loss value: 637.271
Current loss value: 674.152
Filter 192 processed in 13s
Processing filter 193
Current loss value: 0.0
Filter 193 processed in 2s
Processing filter 194
Current loss value: 0.0
Filter 194 processed in 2s
Processing filter 195
Current loss value: 0.0
Filter 195 processed in 2s
Processing filter 196
Current loss 

C:\Users\Yoogie\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:76: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
